In [1]:
import pandas as pd
import numpy as np
import datetime

# Q9

In [2]:
def calculate_distance_matrix(df) -> pd.DataFrame():
    toll_ids = pd.concat([df['id_start'], df['id_end']]).unique()
    distance_matrix = pd.DataFrame(np.inf, index=toll_ids, columns=toll_ids, dtype=float)
    np.fill_diagonal(distance_matrix.values, 0)
    
    for _, row in df.iterrows():
        source = row['id_start']
        destination = row['id_end']
        distance = row['distance']
        distance_matrix.at[source, destination] = distance
        distance_matrix.at[destination, source] = distance

    for k in toll_ids:
        for i in toll_ids:
            for j in toll_ids:
                if distance_matrix.at[i, j] > distance_matrix.at[i, k] + distance_matrix.at[k, j]:
                    distance_matrix.at[i, j] = distance_matrix.at[i, k] + distance_matrix.at[k, j]
    return distance_matrix


In [3]:
df = pd.read_csv('dataset-2.csv')
df.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


In [4]:
distance_matrix = calculate_distance_matrix(df)
print("Distance Matrix:")
print(distance_matrix.head())

Distance Matrix:
         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7     29.9     45.9     67.6     78.7     94.3   
1001402      9.7      0.0     20.2     36.2     57.9     69.0     84.6   
1001404     29.9     20.2      0.0     16.0     37.7     48.8     64.4   
1001406     45.9     36.2     16.0      0.0     21.7     32.8     48.4   
1001408     67.6     57.9     37.7     21.7      0.0     11.1     26.7   

         1001414  1001416  1001418  ...  1001458  1001460  1001461  1001462  \
1001400    112.5    125.7    139.3  ...    348.8    353.9    366.7    371.8   
1001402    102.8    116.0    129.6  ...    339.1    344.2    357.0    362.1   
1001404     82.6     95.8    109.4  ...    318.9    324.0    336.8    341.9   
1001406     66.6     79.8     93.4  ...    302.9    308.0    320.8    325.9   
1001408     44.9     58.1     71.7  ...    281.2    286.3    299.1    304.2   

         1001464  1001466  1001468  1001470  1001437  1001472  

# Q10

In [5]:
def unroll_distance_matrix(df) -> pd.DataFrame:
    unrolled_data = []
    for id_start in df.index:
        for id_end in df.columns:
            if id_start != id_end:
                distance = df.at[id_start, id_end]
                unrolled_data.append([id_start, id_end, distance])
    unrolled_df = pd.DataFrame(unrolled_data, columns=['id_start', 'id_end', 'distance'])
    return unrolled_df

In [6]:
unrolled_df = unroll_distance_matrix(distance_matrix)
print("\nUnrolled Distance Matrix:")
print(unrolled_df)


Unrolled Distance Matrix:
      id_start   id_end  distance
0      1001400  1001402       9.7
1      1001400  1001404      29.9
2      1001400  1001406      45.9
3      1001400  1001408      67.6
4      1001400  1001410      78.7
...        ...      ...       ...
1801   1001472  1001464      45.8
1802   1001472  1001466      37.3
1803   1001472  1001468      26.6
1804   1001472  1001470      16.0
1805   1001472  1001437     202.2

[1806 rows x 3 columns]


# Q11

In [7]:
def find_ids_within_ten_percentage_threshold(df, reference_id) -> pd.DataFrame:
    reference_avg_distance = df[df['id_start'] == reference_id]['distance'].mean()
    threshold_floor = reference_avg_distance * 0.9
    threshold_ceiling = reference_avg_distance * 1.1 
    matching_ids = []
    
    for id_start in df['id_start'].unique():
        avg_distance = df[df['id_start'] == id_start]['distance'].mean()
        if threshold_floor <= avg_distance <= threshold_ceiling:
            matching_ids.append(id_start) 
    return sorted(matching_ids)


In [8]:
reference_id = 1001416
ids_within_threshold = find_ids_within_ten_percentage_threshold(unrolled_df, reference_id)
print("\nIDs within 10% of average distance for reference ID {}:".format(reference_id))
print(ids_within_threshold)


IDs within 10% of average distance for reference ID 1001416:
[1001414, 1001416, 1001418, 1001458, 1001460, 1001461, 1001462]


# Q12

In [9]:
def calculate_toll_rate(df) -> pd.DataFrame:
    df['moto'] = df['distance'] * 0.8
    df['car'] = df['distance'] * 1.2
    df['rv'] = df['distance'] * 1.5
    df['bus'] = df['distance'] * 2.2
    df['truck'] = df['distance'] * 3.6
    return df

In [10]:
toll_df = calculate_toll_rate(unrolled_df)
print("\nToll Rates:")
print(toll_df)


Toll Rates:
      id_start   id_end  distance    moto     car      rv     bus   truck
0      1001400  1001402       9.7    7.76   11.64   14.55   21.34   34.92
1      1001400  1001404      29.9   23.92   35.88   44.85   65.78  107.64
2      1001400  1001406      45.9   36.72   55.08   68.85  100.98  165.24
3      1001400  1001408      67.6   54.08   81.12  101.40  148.72  243.36
4      1001400  1001410      78.7   62.96   94.44  118.05  173.14  283.32
...        ...      ...       ...     ...     ...     ...     ...     ...
1801   1001472  1001464      45.8   36.64   54.96   68.70  100.76  164.88
1802   1001472  1001466      37.3   29.84   44.76   55.95   82.06  134.28
1803   1001472  1001468      26.6   21.28   31.92   39.90   58.52   95.76
1804   1001472  1001470      16.0   12.80   19.20   24.00   35.20   57.60
1805   1001472  1001437     202.2  161.76  242.64  303.30  444.84  727.92

[1806 rows x 8 columns]


# Q13

In [11]:
def calculate_time_based_toll_rates(df) -> pd.DataFrame:
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    time_ranges = [
        (datetime.time(0, 0), datetime.time(10, 0), 0.8),
        (datetime.time(10, 0), datetime.time(18, 0), 1.2),
        (datetime.time(18, 0), datetime.time(23, 59, 59), 0.8)
    ]

    new_rows = []
    for _, row in df.iterrows():
        for day in days:
            for start_time, end_time, weekday_discount in time_ranges:
                new_row = row.copy()
                new_row['start_day'] = day
                new_row['end_day'] = day
                new_row['start_time'] = start_time
                new_row['end_time'] = end_time

                if day in ['Saturday', 'Sunday']:
                    discount = 0.7
                else:
                    discount = weekday_discount

                for vehicle in ['moto', 'car', 'rv', 'bus', 'truck']:
                    new_row[vehicle] *= discount
                new_row['id_start'] = int(new_row['id_start'])
                new_row['id_end'] = int(new_row['id_end'])
                new_rows.append(new_row)
    result_df = pd.DataFrame(new_rows)
    result_df = result_df[['id_start', 'start_time', 'start_day', 'id_end', 'end_time', 'end_day', 'distance', 'moto', 'car', 'rv', 'bus', 'truck']]
    return result_df


In [12]:
time_based_toll_df = calculate_time_based_toll_rates(toll_df)
print("\nTime-Based Toll Rates:")
print(time_based_toll_df)


Time-Based Toll Rates:
      id_start start_time start_day   id_end  end_time   end_day  distance  \
0      1001400   00:00:00    Monday  1001402  10:00:00    Monday       9.7   
0      1001400   10:00:00    Monday  1001402  18:00:00    Monday       9.7   
0      1001400   18:00:00    Monday  1001402  23:59:59    Monday       9.7   
0      1001400   00:00:00   Tuesday  1001402  10:00:00   Tuesday       9.7   
0      1001400   10:00:00   Tuesday  1001402  18:00:00   Tuesday       9.7   
...        ...        ...       ...      ...       ...       ...       ...   
1805   1001472   10:00:00  Saturday  1001437  18:00:00  Saturday     202.2   
1805   1001472   18:00:00  Saturday  1001437  23:59:59  Saturday     202.2   
1805   1001472   00:00:00    Sunday  1001437  10:00:00    Sunday     202.2   
1805   1001472   10:00:00    Sunday  1001437  18:00:00    Sunday     202.2   
1805   1001472   18:00:00    Sunday  1001437  23:59:59    Sunday     202.2   

         moto      car      rv      bus